In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [10]:
df = pd.read_csv("mapped-v2_imputed_mf5iter.csv")
df.shape

(35970, 39)

In [11]:
df['DC201'].value_counts()

DC201
1.0    32398
0.0     3572
Name: count, dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35970 entries, 0 to 35969
Data columns (total 39 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DC216   35970 non-null  float64
 1   DC220   35970 non-null  float64
 2   DC142a  35970 non-null  float64
 3   DC024   35970 non-null  int64  
 4   DC025   35970 non-null  int64  
 5   DC205   35970 non-null  float64
 6   DC206   35970 non-null  float64
 7   DC207   35970 non-null  float64
 8   DC208   35970 non-null  float64
 9   DC209   35970 non-null  float64
 10  DC210   35970 non-null  float64
 11  DC211   35970 non-null  float64
 12  DC212   35970 non-null  float64
 13  DC213   35970 non-null  float64
 14  DC214   35970 non-null  float64
 15  DC215   35970 non-null  float64
 16  DC217   35970 non-null  int64  
 17  DC219   35970 non-null  int64  
 18  DC226   35970 non-null  float64
 19  DC230a  35970 non-null  float64
 20  DC230b  35970 non-null  float64
 21  DC232   35970 non-null  float64
 22

In [15]:
# # Use Seaborn to make a correlation matrix

# corr_matrix = df.corr()
# fig, ax = plt.subplots(figsize=(25,25))
# ax = sns.heatmap(corr_matrix,
#                 annot=True,
#                 linewidths=0.5,
#                 fmt=".2f",
#                 cmap="YlGnBu");

In [7]:
target = "DC201"

In [8]:
y_train = np.array(df[target]) # value dari label
df = df.drop(target, axis=1) # drop target
feature_list = list(df.columns) # list column
X_train = np.array(df)

SMOTE

In [11]:
# from imblearn.over_sampling import SMOTE
# from sklearn.datasets import make_classification


# # Melakukan oversampling dengan SMOTE
# smote = SMOTE()
# X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# # Melihat jumlah sampel setelah oversampling
# print("Jumlah sampel sebelum oversampling:", len(X_train))
# print("Jumlah sampel setelah oversampling:", len(X_resampled))

Jumlah sampel sebelum oversampling: 35973
Jumlah sampel setelah oversampling: 64802


In [16]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)

In [17]:
X_train_norm

array([[0.05      , 0.78823529, 0.06012024, ..., 0.5       , 0.        ,
        0.125     ],
       [0.1       , 0.36470588, 0.07014028, ..., 0.25      , 0.25      ,
        0.375     ],
       [0.1       , 0.25882353, 0.0751503 , ..., 0.        , 0.75      ,
        0.        ],
       ...,
       [0.1       , 0.45882353, 0.04809619, ..., 0.25      , 0.        ,
        0.        ],
       [0.1       , 0.52941176, 0.03907816, ..., 0.        , 0.25      ,
        0.        ],
       [0.05      , 0.21176471, 0.04008016, ..., 0.        , 0.5       ,
        0.        ]])

In [18]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.2
RANDOM_SEED = 36

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train_norm, y_train, test_size = TEST_SIZE, random_state = RANDOM_SEED)

In [23]:
from lightgbm import LGBMClassifier
import lightgbm.plotting as lgbm_plotting

model = LGBMClassifier(learning_rate=0.07, 
                       max_depth=20, 
                       random_state=40, 
                       num_leaves=100,
                       bagging_freq=1,
                       min_data_in_leaf=20,
                       feature_fraction =1)
model.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)],
          verbose=22, eval_metric='l1')

/home/codespace/.python/current/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[22]	training's l1: 0.132746	training's binary_logloss: 0.199951	valid_0's l1: 0.141904	valid_0's binary_logloss: 0.226178
[44]	training's l1: 0.11463	training's binary_logloss: 0.167457	valid_0's l1: 0.131008	valid_0's binary_logloss: 0.216025
[66]	training's l1: 0.104997	training's binary_logloss: 0.146804	valid_0's l1: 0.127522	valid_0's binary_logloss: 0.215562
[88]	training's l1: 0.097014	training's binary_logloss: 0.131112	valid_0's l1: 0.125112	valid_0's binary_logloss: 0.216081


LGBMClassifier(bagging_freq=1, feature_fraction=1, learning_rate=0.07,
               max_depth=20, min_data_in_leaf=20, num_leaves=100,
               random_state=40)

In [24]:
print('Training accuracy {:.4f}'.format(model.score(X_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(X_test,y_test)))

Training accuracy 0.9537
Testing accuracy 0.9190


In [25]:
from sklearn.metrics import classification_report

print(classification_report(y_test,model.predict(X_test),digits=5))

              precision    recall  f1-score   support

         0.0    0.67771   0.32097   0.43562       701
         1.0    0.93063   0.98352   0.95635      6493

    accuracy                        0.91896      7194
   macro avg    0.80417   0.65225   0.69599      7194
weighted avg    0.90599   0.91896   0.90561      7194

